In [2]:
import cv2
print(cv2.__version__)

4.1.1


In [19]:
#Required Libraries
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
import glob #For reading the files
import os #For creating directories

unseen_test_videos = 1 # set to 1 if you are splitting frames for unseen test videos, 0 for training videos

#Reading Videos
#This only gets the file names
if unseen_test_videos == 0:
    main_folder = "Training_Videos/*"
    outputFrames_folder = 'New_Training_Video_Frames/'
else:
    main_folder = "Unseen_Test_Videos/*"
    outputFrames_folder = 'Unseen_Test_Video_Frames/'
videos = glob.glob(main_folder)
print("Number of videos to split into frames:",len(videos))

#Dictionaries for NN and to keep track of the vides which have already been broken down into frames
dict_exercise = {'BenchPress':0,'Deadlift':1,'Squat':2,'Military Press':3}
inverted_dict_exercise = dict([[v,k] for k,v in dict_exercise.items()])
#exercise_frames_count = {'Bench Press':0,'Deadlift':0,'Squat':0,'Military Press':0}

dictionary_videos_done_good = {'BenchPress':[],'Deadlift':[],'Squat':[],'Military Press':[]}
dictionary_videos_done_bad = {'BenchPress':[],'Deadlift':[],'Squat':[],'Military Press':[]}

for video in videos:

    #default video number
    video_number = -5
    filename = video.split('/')[-1]
    index = filename.index('_')
    
    #Getting the second part of the file name after split
    form_part = filename.split('_')[1]
    folder_name = filename.split('.')[0]
    
    #print("Video:",filename)
    #print("Form Part:",form_part)
    #print('Directory Name:',directory_name)

    #Obtaining indices to check whether a rep was done in good/bad form
    if '(' in form_part:
        #Index of '('
        index_post_form = form_part.index('(')
    else:
        index_post_form = form_part.index('.')
    
    #Obtaining rep number
    try:
        index_bracket_open = filename.index('(')
    
        index_bracket_close = filename.index(')')
    except:
        video_number = 0

    if video_number!=0:
        video_number = int(filename[index_bracket_open+1:index_bracket_close])

    #Exercise name
    exercise = filename[:index]
    #print(exercise)
    
    #Good/Bad form
    form_good_bad = form_part[:index_post_form]
    form_good_bad = form_good_bad.lstrip()
    #print("THIS:",form_good_bad)
    
#     sub_directory = 'New_Training_Video_Frames/'+exercise
    sub_directory = outputFrames_folder+exercise
    if not os.path.exists(sub_directory):
            os.makedirs(sub_directory)
    
    if form_good_bad == 'Good':
        #print("Dictionary Videos done list for good:",dictionary_videos_done_good[exercise])
        if video_number not in dictionary_videos_done_good[exercise]:
            frame_number=0
    else:
        #print("Dictionary Videos done list for bad:",dictionary_videos_done_bad[exercise])
        if video_number not in dictionary_videos_done_bad[exercise]:
            frame_number=0

    vidcap = cv2.VideoCapture(video)
    success,image = vidcap.read()
    #count = 0
    fps_multiplier = 100 # increase this number to decrease the fps (i.e 100 = 9fps, 1000 = 1fps)
    success = True
    folder_name = sub_directory+'/'+folder_name
    #print(folder_name)
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(frame_number*fps_multiplier)) # caps the frame rate
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        filename = folder_name+"/"+exercise+"_"+form_good_bad+"_"+str(video_number)+"_"+str(frame_number)+".jpg"
        cv2.imwrite(filename, image)     # save frame as JPEG file
        success,image = vidcap.read()
        #print(filename)
        #print('Read a new frame: ', success)
        frame_number += 1
    
    if form_good_bad == 'Good':
        dictionary_videos_done_good[exercise].append(video_number)
    else:
        dictionary_videos_done_bad[exercise].append(video_number)
print ("Done!")

Number of videos to split into frames: 2
Done!
